## Подготовка данных для занятия

In [2]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
28,0,и,7
36,0,который,5
110,0,шерлок,4
83,0,сериал,4
76,0,с,4
8,0,весь,3
74,0,риколетти,3
104,0,холмс,2
22,0,же,2


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [30]:
# можем взять за основу уже сформированный word_count_df с группировкой
doc_frequency = pd.DataFrame(word_count_df.groupby(['word'])['doc_id'].count().reset_index())
# doc_frequency.sort_values(by='doc_id', ascending=False)

# или бызовый tokens_df
doc_frequency = pd.DataFrame(tokens_df.groupby(['word'])['doc_id'].nunique().reset_index())
# doc_frequency.sort_values(by='doc_id', ascending=False)

doc_frequency['doc_count'] = doc_count
doc_frequency['d'] = round((doc_frequency['doc_id'] / doc_frequency['doc_count']) * 100)

doc_frequency.sort_values(by='doc_id', ascending=False)

,word,doc_id,doc_count,d
172,и,8,8,100.0
41,в,8,8,100.0
270,на,7,8,88.0
468,с,7,8,88.0
324,он,6,8,75.0
...,...,...,...,...
233,леди,1,8,12.0
234,лестрейд,1,8,12.0
235,личико,1,8,12.0
236,лишаться,1,8,12.0


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [36]:

raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
# Предполагаем, что имён и фамилий, состоящих из одной буквы не бывает, минимум из двух. 
# Хотя бы обрежем однобуквенные предлоги и союзы в начале предложений.
# Можно было бы получить словарь предлогов и союзов и исключить их из пар.
reg_expr = r'[A-Я]+\w [A-Я]+\w+'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Стивена Фрая
Джоном Дженксом
У Теда
Теперь Тед
Вскоре Тед


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [62]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])


doc_to_genres = word_count_df.merge(genres_df, how='inner', left_on='word', right_on='character_word')
funcs = lambda x: x.drop_duplicates().values.tolist()
doc_to_genres = doc_to_genres[['doc_id', 'genre']].groupby(['doc_id']).agg(funcs)
doc_to_genres

,genre
doc_id,
0,"[детектив, триллер]"
1,[сказка]
2,[мелодрама]
3,[комедия]
4,[комедия]
6,[триллер]
